# Synthetic Traces Plotting/Analysis

The real traces are taken from a dataset found on Kaggle (and GitHub). During the experiment, there are also synthetic traces that are generated on the fly based on some distribution or function. This notebook explores the latter.

Note that there are some limitations:

* Each episode has a length of 288 steps (each step represents 5 minutes in a 24 hour period).
* In each step, there can be a maximum of 150 incoming requests to a node.

In [ ]:
# Common imports.
from pathlib import Path

%matplotlib widget
import base

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets

## Sinusoidal traces

## Generation method

In [ ]:
rng = np.random.default_rng(seed=42)  # RNG used to generate the requests.
min_reqs, max_reqs = 0, 150  # Range of the generated requests.
max_steps = 288  # Steps of a single episode (see the DFaaS env).

In [ ]:
# Generates the requests for a single agent.
def synthetic_sinusoidal(
    average_requests=50,
    amplitude_requests=100,
    noise_ratio=0.1,
    unique_periods=3,
    seed=0,
):
    steps = np.arange(max_steps)

    rng = np.random.default_rng(seed=seed)

    # The periods changes unique_periods times. By default is 3, so every 96 steps
    # (max_steps = 288). We first generate the periods and expand the array to match
    # the max_steps. If max_steps is not a multiple of 96, some elements must be
    # appended at the end, hence the resize call.
    repeats = max_steps // unique_periods
    periods = rng.uniform(15, high=100, size=unique_periods)
    periods = np.repeat(periods, repeats)  # Expand the single values.
    periods = np.resize(periods, periods.size + max_steps - periods.size)

    base_input = average_requests + amplitude_requests * np.sin(
        2 * np.pi * steps / periods
    )
    noisy_input = base_input + noise_ratio * rng.normal(
        0, amplitude_requests, size=max_steps
    )
    requests = np.asarray(noisy_input, dtype=np.int32)

    # Clip the excess values respecting the minimum and maximum values
    # for the input requests observation.
    np.clip(requests, min_reqs, max_reqs, out=requests)

    return requests

### Plotting of a generic trace

In [ ]:
def plot_sinusoidal_trace():
    avg_reqs = ipywidgets.IntSlider(
        value=50,
        min=0,
        max=200,
        description="Average requests [0, 200]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    ampl_reqs = ipywidgets.IntSlider(
        value=100,
        min=0,
        max=200,
        description="Amplitude requests [0, 200]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    noise = ipywidgets.FloatSlider(
        value=0.1,
        min=0,
        max=2,
        description="Noise [0, 2]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    unique_periods = ipywidgets.IntSlider(
        value=3,
        min=1,
        max=5,
        description="Unique periods [1, 5]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    seed = ipywidgets.BoundedIntText(
        value=42,
        min=0,
        max=np.iinfo(np.uint32).max,
        description="Seed:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )

    with plt.ioff():
        fig = plt.figure(layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    def make_plot():
        ax.clear()

        trace = synthetic_sinusoidal(
            average_requests=avg_reqs.value,
            amplitude_requests=ampl_reqs.value,
            noise_ratio=noise.value,
            unique_periods=unique_periods.value,
            seed=seed.value,
        )

        step_idx = np.arange(1, max_steps + 1)
        ax.bar(step_idx, trace)

        ax.set_title(f"Function invocations")
        ax.set_ylabel("Invocations")
        ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
        ax.set_ylim(bottom=min_reqs, top=max_reqs + 5)
        ax.set_xlabel("Step")
        ax.xaxis.set_major_locator(ticker.MultipleLocator(25))

        ax.grid(axis="both")
        ax.set_axisbelow(True)

        fig.canvas.draw_idle()
        fig.canvas.flush_events()

    # Make the initial plot with the default values.
    make_plot()

    # Link the input widgets to the plotting function.
    avg_reqs.observe(lambda change: make_plot(), names="value")
    ampl_reqs.observe(lambda change: make_plot(), names="value")
    noise.observe(lambda change: make_plot(), names="value")
    unique_periods.observe(lambda change: make_plot(), names="value")
    seed.observe(lambda change: make_plot(), names="value")

    # Callback when the user press the reset button.
    def reset(widget, default_value):
        widget.value = default_value
        make_plot()

    # Create the reset buttons and link them to the correct widget.
    avg_reqs_reset = ipywidgets.Button(description="Reset", icon="undo")
    avg_reqs_reset.on_click(lambda _: reset(avg_reqs, 50))
    ampl_reqs_reset = ipywidgets.Button(description="Reset", icon="undo")
    ampl_reqs_reset.on_click(lambda _: reset(ampl_reqs, 100))
    noise_reset = ipywidgets.Button(description="Reset", icon="undo")
    noise_reset.on_click(lambda _: reset(noise, 0.1))
    unique_periods_reset = ipywidgets.Button(description="Reset", icon="undo")
    unique_periods_reset.on_click(lambda _: reset(unique_periods, 3))

    avg_reqs_reset.click()

    # Create the last button to generate a random seed.
    def random_seed():
        iinfo = np.iinfo(np.uint32)
        seed.value = rng.integers(0, high=iinfo.max, size=1)[0].item()

    random_seed_btn = ipywidgets.Button(description="Random", icon="shuffle")
    random_seed_btn.on_click(lambda _: random_seed())

    inputs = ipywidgets.VBox(
        [
            ipywidgets.HBox([avg_reqs, avg_reqs_reset]),
            ipywidgets.HBox([ampl_reqs, ampl_reqs_reset]),
            ipywidgets.HBox([noise, noise_reset]),
            ipywidgets.HBox([unique_periods, unique_periods_reset]),
            ipywidgets.HBox([seed, random_seed_btn]),
        ]
    )

    return ipywidgets.AppLayout(
        header=inputs,
        center=fig.canvas,
        pane_heights=[1, 5, 0],
    )


plot_sinusoidal_trace()